<a href="https://colab.research.google.com/github/BishalMondal/Crop-Disease-Detection/blob/main/InceptionV3_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
print(keras.__version__)

2.7.0


In [2]:
pip install pyyaml h5py

In [ ]:
#run only if mounting drive to be done
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [3]:
def loadTrainData(image_resize, batch_size_training):
  train_generator = data_generator.flow_from_directory(
    "/content/drive/MyDrive/training-data",
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')
  return train_generator

In [4]:
def loadValidationData(image_resize, batch_size_validation):
  validation_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/validation-data',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')
  return validation_generator

In [14]:
def myCallback():
  checkpoint_filepath = '/content/drive/MyDrive/tmp/checkpoint/'

  #checkpoint_dir = os.path.dirname(checkpoint_filepath)

  try:
    os.mkdir(checkpoint_filepath)
  except OSError as error:
    print(error)    
  model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


  return checkpoint_filepath, model_checkpoint_callback

In [15]:
def get_model():
  model = Sequential()

  model.add(InceptionV3(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
  
  num_classes = 61
  model.add(Dense(num_classes, activation='softmax'))

  model.layers[0].trainable = False

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [16]:
def train_model(epoch_num, image_resize, batch_size_training, batch_size_validation, model_checkpoint_callback, checkpoint_filepath):
  num_classes = 61

  #image_resize = 224

  #batch_size_training = 64
  #batch_size_validation = 64
  train_generator = loadTrainData(image_resize, batch_size_training)
  validation_generator = loadValidationData(image_resize, batch_size_validation)

  model = get_model()

  fit_history = model.fit(
    train_generator,
    steps_per_epoch=batch_size_training/4,
    epochs=epoch_num,
    validation_data=validation_generator,
    validation_steps=batch_size_validation/4,
    callbacks=[model_checkpoint_callback]
  )

  #model.load_weights(checkpoint_filepath)
  #!mkdir -p saved_model
  model.save('saved_model/my_model')

  return fit_history

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
checkpoint_filepath, model_checkpoint_callback = myCallback()
os.listdir(checkpoint_filepath)

[Errno 17] File exists: '/content/drive/MyDrive/tmp/checkpoint/'


[]

In [19]:
fit_history = train_model(20, 224, 64, 64, model_checkpoint_callback, checkpoint_filepath)

Found 31751 images belonging to 61 classes.
Found 4546 images belonging to 61 classes.
Epoch 1/20
16/16 [==============================] - 528s 34s/step - loss: 17.2072 - accuracy: 0.0713 - val_loss: 13.7332 - val_accuracy: 0.0811
Epoch 2/20
16/16 [==============================] - 440s 28s/step - loss: 11.0885 - accuracy: 0.1289 - val_loss: 10.3593 - val_accuracy: 0.1406
Epoch 3/20
16/16 [==============================] - 393s 25s/step - loss: 9.0805 - accuracy: 0.1717 - val_loss: 8.4138 - val_accuracy: 0.2207
Epoch 4/20
16/16 [==============================] - 367s 23s/step - loss: 7.8577 - accuracy: 0.2080 - val_loss: 7.5199 - val_accuracy: 0.2188
Epoch 5/20
16/16 [==============================] - 339s 21s/step - loss: 7.8436 - accuracy: 0.2246 - val_loss: 7.4073 - val_accuracy: 0.2178
Epoch 6/20
16/16 [==============================] - 302s 19s/step - loss: 7.9664 - accuracy: 0.2227 - val_loss: 6.9401 - val_accuracy: 0.2002
Epoch 7/20
16/16 [==============================] - 294s 